# Missing Dates

In [1]:
import os
from tqdm import tqdm
from datetime import date, timedelta

train_dates=[]
valid_dates=[]
flag=0

for filename in tqdm(os.listdir('Data/tif files')):
    temp_date=str(filename[-12:-8])+'-'+str(filename[-8:-6])+'-'+str(filename[-6:-4])
    if flag==0:
    	train_dates.append(temp_date)
    else :
    	valid_dates.append(temp_date)
    if filename == 'SWI_SMAP_I_20200327_20200329.tif':
    	flag = 1
        
train_dates_list = [date(int(i[0:4]),int(i[5:7]),int(i[8:])) for i in train_dates]
valid_dates_list = [date(int(i[0:4]),int(i[5:7]),int(i[8:])) for i in valid_dates]

100%|██████████████████████████████████████████████████████████████████████████| 1776/1776 [00:00<00:00, 891038.74it/s]


In [2]:
start_date = train_dates_list[0]
missing_train_dates=[]
one_day = timedelta(days=1)
while start_date < train_dates_list[-1]:
    if start_date not in train_dates_list:
        missing_train_dates.append(start_date)
    start_date += one_day
    
train_dates_list = [str(i) for i in train_dates_list]
missing_train_dates = [str(i) for i in missing_train_dates]
train_dates_list = train_dates_list + missing_train_dates

train_dates_list.sort()

In [3]:
start_date = valid_dates_list[0]
missing_valid_dates=[]
one_day = timedelta(days=1)
while start_date < valid_dates_list[-1]:
    if start_date not in valid_dates_list:
        missing_valid_dates.append(start_date)
    start_date += one_day
    
valid_dates_list = [str(i) for i in valid_dates_list]
missing_valid_dates = [str(i) for i in missing_valid_dates]
valid_dates_list = valid_dates_list + missing_valid_dates

valid_dates_list.sort()

In [4]:
import pickle
with open('train_dates.pkl', 'wb') as picklefile:
    pickle.dump(train_dates_list, picklefile)
with open('valid_dates.pkl', 'wb') as picklefile:
    pickle.dump(valid_dates_list, picklefile)

# Loading  Images

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
import skimage
from skimage.io import imread
from tqdm import tqdm
from datetime import date, timedelta

train_data,valid_data = [],[]
flag=0
missing = np.full([256,272,1], np.nan)

train_start = date(2015,4,4)
train_end   = date(2020,3,29)
valid_start = date(2020,5,24)
valid_end   = date(2020,10,9)
one_day = timedelta(days=1)

for filename in tqdm(os.listdir('Data/tif files')):
    # print(filename)
    name= str(filename[-12:-8])+'-'+str(filename[-8:-6])+'-'+str(filename[-6:-4])
    current = date(int(name[0:4]),int(name[5:7]),int(name[8:]))
    if flag==0:
        if current != train_start:
            train_data.extend([missing]*int((current - train_start).days))
            train_start = current
    else:
        if current != valid_start:
            valid_data.extend([missing]*int((current - valid_start).days))
            valid_start = current
    img = imread('Data/tif files/'+filename)
    img = np.expand_dims(img,axis=-1)
    img = np.asarray(img)
    if flag==0:
        train_data.append(img)
        train_start += one_day    
    else :
        valid_data.append(img)
        valid_start += one_day
    if filename == 'SWI_SMAP_I_20200327_20200329.tif':
        flag = 1
train_data=np.asarray(train_data)
valid_data=np.asarray(valid_data)
data = np.concatenate((train_data,valid_data),axis=0)
print(train_data.shape)
print(valid_data.shape)
print(data.shape)

with open('Data/data.pkl', 'wb') as picklefile:
    pickle.dump(data, picklefile)
with open('Data/train_data.pkl', 'wb') as picklefile:
    pickle.dump(train_data, picklefile)
with open('Data/valid_data.pkl', 'wb') as picklefile:
    pickle.dump(valid_data, picklefile)

100%|██████████████████████████████████████████████████████████████████████████████| 1776/1776 [00:43<00:00, 40.45it/s]


(1822, 256, 272, 1)
(139, 256, 272, 1)
(1961, 256, 272, 1)


# Mask Data

In [6]:
import pandas as pd
import numpy as np
import skimage
from skimage.io import imread
from tqdm import tqdm
import os
import pickle

with open('Data/train_data.pkl', 'rb') as picklefile:
    train_data=pickle.load(picklefile)
with open('Data/valid_data.pkl', 'rb') as picklefile:
    valid_data=pickle.load(picklefile)

train_data_nan = np.isnan(train_data)

mask = train_data_nan[0]

for i in train_data_nan:
	mask = mask & i

mask_train_data,mask_valid_data = train_data,valid_data
for i,a in enumerate(train_data):
    a[mask==1]=0 
    mask_train_data[i]=a

mask_valid_data = mask_valid_data
for i,a in enumerate(valid_data):
    a[mask==1]=0 
    mask_valid_data[i]=a

mask_data=np.concatenate((mask_train_data,mask_valid_data),axis=0)
with open('Data/masked_data.pkl', 'wb') as picklefile:
    pickle.dump(mask_data, picklefile)
with open('Data/masked_train_data.pkl', 'wb') as picklefile:
    pickle.dump(mask_train_data, picklefile)
with open('Data/masked_valid_data.pkl', 'wb') as picklefile:
    pickle.dump(mask_valid_data, picklefile)


# Imputing Missing Values

In [7]:
import pandas as pd
import numpy as np
import skimage
from skimage.io import imread
from tqdm import tqdm
import os
import pickle


with open('Data/masked_valid_data.pkl', 'rb') as picklefile:
    mask_valid_data=pickle.load(picklefile)

with open('Data/masked_train_data.pkl', 'rb') as picklefile:
    mask_train_data=pickle.load(picklefile)

mask_train_data = np.reshape(mask_train_data,(len(mask_train_data),256*272))
mask_valid_data = np.reshape(mask_valid_data,(len(mask_valid_data),256*272))

imputed_train_data,imputed_valid_data = mask_train_data,mask_valid_data

for i in tqdm(range(256*272)):
    series= pd.Series(imputed_train_data[:,i])
    series = series.interpolate(method='linear')
    series = series.fillna(method='ffill')
    series = series.fillna(method='bfill')
    series = series.fillna(0)
    imputed_train_data[:,i]=series.values

for i in tqdm(range(256*272)):
    series= pd.Series(imputed_valid_data[:,i])
    series = series.interpolate(method='linear')
    series = series.fillna(method='ffill')
    series = series.fillna(method='bfill')
    series = series.fillna(0)
    imputed_valid_data[:,i]=series.values

imputed_train_data = imputed_train_data.reshape(len(imputed_train_data),256,272,1)
imputed_valid_data = imputed_valid_data.reshape(len(imputed_valid_data),256,272,1)
imputed_data = np.concatenate((imputed_train_data,imputed_valid_data),axis=0)

with open('Data/imputed_data.pkl', 'wb') as picklefile:
    pickle.dump(imputed_data, picklefile)

with open('Data/imputed_train_data.pkl', 'wb') as picklefile:
    pickle.dump(imputed_train_data, picklefile)

with open('Data/imputed_valid_data.pkl', 'wb') as picklefile:
    pickle.dump(imputed_valid_data, picklefile)

100%|██████████████████████████████████████████████████████████████████████████| 69632/69632 [00:45<00:00, 1528.72it/s]
